In [1]:

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1" 
import tensorflow as tf
print("GPU available:", tf.config.list_physical_devices('GPU'))
from tensorflow import keras
import time
import os
import json
import importlib
from matplotlib import pyplot as plt
import preprocess
# import preprocess as prep
import SLR_model
import SLR_model_GRU
import numpy as np
# from preprocess import person



# model will output multiple(5) results/sec, how are we gonna handle it?


2024-11-06 17:04:43.236296: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-06 17:04:43.242038: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 17:04:43.249073: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 17:04:43.251311: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 17:04:43.256749: I tensorflow/core/platform/cpu_feature_guar

GPU available: []


2024-11-06 17:04:43.900361: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-06 17:04:43.900383: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:135] retrieving CUDA diagnostic information for host: shdUbuntu
2024-11-06 17:04:43.900386: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:142] hostname: shdUbuntu
2024-11-06 17:04:43.900427: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:166] libcuda reported version is: 560.35.3
2024-11-06 17:04:43.900439: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:170] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  560.35.03  Release Build  (dvs-builder@U16-I1-N07-12-3)  Fri Aug 16 21:42:42 UTC 2024
GCC version:  gcc version 13.2.0 (Ubuntu 13.2.0-23ubuntu

In [2]:
load_size = 3000 # number of data to be loaded at once
epochs = 50
run_time=2
batch_size = 16
save_dir = "saves_GRU"
load_dir = "saves_GRU"
model = SLR_model_GRU.get_model()

In [ ]:
 # reload model file
end_file=preprocess.getoutputdir()

save_suffix = time.strftime("%m-%d-%H", time.localtime(time.time()))
ckpt_name=save_suffix+"-"+str(epochs)+"epochs-"+str(run_time)+"times"

check_path = os.path.join(save_dir,'ckpt',ckpt_name)
hist_path = os.path.join(save_dir, "hist",ckpt_name+".json")

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
filepath=f'{check_path}.keras',
monitor='categorical_accuracy',
mode='max',
save_freq='epoch',
save_best_only=True)


start_person=1
start_count=1
start_word=" "
with open(os.path.join('logs',ckpt_name+'.txt'), 'a') as logs:
    for i in range(1,17):
        l_raws=[]
        r_raws=[]
        p_raws=[]
        y_raws=[]
        loss_weights_raws=[]
        if start_person>i:
            continue
        for k in range(1,run_time+1):
            if start_count>k:
                continue
            elif start_count==k:
                start_count=0
            for j in sorted(os.listdir(os.path.join(preprocess.getoutputdir(),str(i)))):
                if len(l_raws)==0:
                    start_word = j
                else:
                    end_word = j
                l_raw, r_raw, p_raw, y_raw, loss_weights_raw = preprocess.load_data(f"{i}/{j}")
                l_raws.append(l_raw)
                r_raws.append(r_raw)
                p_raws.append(p_raw)
                y_raws.append(y_raw)
                loss_weights_raws.append(loss_weights_raw)
                # print(p_raw.shape)
                # break
                if len(l_raws)>=load_size:
                    
                    logs.write(f'{ time.strftime("%H-%M-%S", time.localtime(time.time()))}:{k}) person:{i} : {start_word} ~ {end_word}\n')  # 한 줄 쓰기
                    l_train, each = SLR_model_GRU.serialize(l_raws)
                    r_train, each = SLR_model_GRU.serialize(r_raws)
                    p_train, each, sample_weights = SLR_model_GRU.serialize(p_raws, stride=2, loss_weights_list=loss_weights_raws)
                    x_train = (l_train, r_train, p_train)
                    
                    y_train = np.repeat(y_raws, each)
                    y_train = SLR_model_GRU.encode_onehot2d(y_train)
                    
                    dataset = SLR_model_GRU.convert_to_dataset(x_train, y_train, batch_size, sample_weights)
                    hist = model.fit(dataset, epochs=epochs, callbacks=[model_checkpoint_callback])
                    
                    # model.save(os.path.join(save_dir, 'model', f"{ckpt_name}.h5"))

                    # converter = tf.lite.TFLiteConverter.from_keras_model(model)
                    # tflite_model = converter.convert()
                    # with open(os.path.join(save_dir, 'model', f"{ckpt_name}.tflite"), 'wb') as tflite_file:
                    #     tflite_file.write(tflite_model)

                    with open(hist_path, 'w') as file:
                        json.dump(hist.history, file)
                    l_raws.clear()
                    r_raws.clear()
                    p_raws.clear()
                    y_raws.clear()
                    loss_weights_raws.clear()
            if len(l_raws)>0:
                l_train, each = SLR_model_GRU.serialize(l_raws)
                r_train, each = SLR_model_GRU.serialize(r_raws)
                p_train, each, sample_weights = SLR_model_GRU.serialize(p_raws, stride=2, loss_weights_list=loss_weights_raws)
                x_train = (l_train, r_train, p_train)
                
                y_train = np.repeat(y_raws, each)
                y_train = SLR_model_GRU.encode_onehot2d(y_train)
                
                
                dataset = SLR_model_GRU.convert_to_dataset(x_train, y_train, batch_size, sample_weights)
                hist = model.fit(dataset, epochs=epochs, callbacks=[model_checkpoint_callback])
                
                # model.save(os.path.join(save_dir, 'model', f"{ckpt_name}.h5"))

                # converter = tf.lite.TFLiteConverter.from_keras_model(model)
                # tflite_model = converter.convert()
                # with open(os.path.join(save_dir, 'model', f"{ckpt_name}.tflite"), 'wb') as tflite_file:
                #     tflite_file.write(tflite_model)

                with open(hist_path, 'w') as file:
                    json.dump(hist.history, file)



Epoch 1/50
3691/3691 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - categorical_accuracy: 0.0014 - loss: 2.2689
Epoch 2/50
3691/3691 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - categorical_accuracy: 0.0018 - loss: 2.2203
Epoch 3/50
2924/3691 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - categorical_accuracy: 0.0015 - loss: 2.1683

In [3]:
# # Configurations
# load_size = 3000 # number of data to be loaded at once
# epochs = 50
# batch_size = 16
# save_dir = "saves"
# load_dir = "saves"
# # load_path = "C:/Users/jerry/Desktop/hly/2024-2/cap/BN_SLR/BomNae-SLR/checkpoints.keras"


In [4]:
# model = SLR_model.get_model()

In [5]:
#  # reload model file
# importlib.reload(SLR_model)
# importlib.reload(preprocess)
# end_file=preprocess.getoutputdir()

# save_suffix = time.strftime("_%d-%H-%M", time.localtime(time.time()))


# # checkTime="_14-16-16"
# # check_path = os.path.join(save_dir, "check"+checkTime)
# # hist_path = os.path.join(save_dir, "hist"+checkTime+".json")

# check_path = os.path.join(save_dir, "check"+save_suffix+"_50epochs_3time")
# hist_path = os.path.join(save_dir, "hist"+save_suffix+".json")

# model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
# filepath=f'{check_path}.keras',
# monitor='categorical_accuracy',
# mode='max',
# save_freq='epoch',
# save_best_only=True)


# start_person=1
# start_count=1
# start_word=" "
# for i in range(1,17):
#     l_raws=[]
#     r_raws=[]
#     p_raws=[]
#     y_raws=[]
#     loss_weights_raws=[]
#     if start_person>i:
#         continue
#     for k in range(1,2):
#         if start_count>k:
#             continue
#         elif start_count==k:
#             start_count=0
#         for j in sorted(os.listdir(os.path.join(preprocess.getoutputdir(),str(i)))):
#             if len(l_raws)==0:
#                 start_word = j
#             else:
#                 end_word = j
#             l_raw, r_raw, p_raw, y_raw, loss_weights_raw = preprocess.load_data(f"{i}/{j}")
#             l_raws.append(l_raw)
#             r_raws.append(r_raw)
#             p_raws.append(p_raw)
#             y_raws.append(y_raw)
#             loss_weights_raws.append(loss_weights_raw)
    
#             if len(l_raws)>=load_size:
#                 with open(str(i)+'training_log.txt', 'a') as logs:
#                     logs.write(f'{ time.strftime("%H-%M-%S", time.localtime(time.time()))}:{k}) person:{i} : {start_word} ~ {end_word}\n')  # 한 줄 쓰기
#                 print(f"{k}) person:{i} : {start_word} ~ {end_word}\n\n\n\n")
#                 l_train, each = SLR_model.serialize(l_raws)
#                 r_train, each = SLR_model.serialize(r_raws)
#                 p_train, each, sample_weights = SLR_model.serialize(p_raws, stride=2, loss_weights_list=loss_weights_raws)
#                 x_train = (l_train, r_train, p_train)
                
#                 y_train = np.repeat(y_raws, each)
#                 y_train = SLR_model.encode_onehot2d(y_train)
    
#                 dataset = SLR_model.convert_to_dataset(x_train, y_train, batch_size, sample_weights)
#                 hist = model.fit(dataset, epochs=epochs, callbacks=[model_checkpoint_callback])
                
#                 with open(hist_path, 'w') as file:
#                     json.dump(hist.history, file)
#                 l_raws.clear()
#                 r_raws.clear()
#                 p_raws.clear()
#                 y_raws.clear()
#                 loss_weights_raws.clear()
#         if len(l_raws)>0:
#             l_train, each = SLR_model.serialize(l_raws)
#             r_train, each = SLR_model.serialize(r_raws)
#             p_train, each, sample_weights = SLR_model.serialize(p_raws, stride=2, loss_weights_list=loss_weights_raws)
#             x_train = (l_train, r_train, p_train)
            
#             y_train = np.repeat(y_raws, each)
#             y_train = SLR_model.encode_onehot2d(y_train)
    
#             dataset = SLR_model.convert_to_dataset(x_train, y_train, batch_size, sample_weights)
#             hist = model.fit(dataset, epochs=epochs, callbacks=[model_checkpoint_callback])
            
#             with open(hist_path, 'w') as file:
#                 json.dump(hist.history, file)

    

In [6]:
# hand input shape: (batch, time, h, w, channels)
# pose input shape: (batch, time, channel, features)

In [7]:
# # 세이브된 모델 로드
# load_path = os.join(load_dir, "check_00-00-00.keras")
# model = SLR_model.load_model(load_path)

In [8]:
# # 안될때 (강제중지 + 초기화)
# tf.keras.backend.clear_session()

In [9]:
# l_raws=[]
# r_raws=[]
# p_raws=[]
# y_raws=[]


# save_suffix = time.strftime("_%d-%H-%M", time.localtime(time.time()))
# check_path = os.path.join(save_dir, "check"+save_suffix)
# hist_path = os.path.join(save_dir, "hist"+save_suffix+".json")

# model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
#     filepath=f'{check_path}.keras',
#     monitor='binary_accuracy',
#     mode='max',
#     save_freq='epoch',
#     save_best_only=True)



# l_raw, r_raw, p_raw, y_raw = preprocess.load_data(f"1/1")

# l_raws.append(l_raw)
# r_raws.append(r_raw)
# p_raws.append(p_raw)
# y_raws.append(y_raw)


# l_train, each = SLR_model.serialize(l_raws)
# r_train, each = SLR_model.serialize(r_raws)
# p_train, each = SLR_model.serialize(p_raws, stride=2)
# y_train = np.repeat(y_raws, each)
# y_train = SLR_model.num_arr2bin(y_train, 12)
# x_train = (l_train, r_train, p_train)

# # dataset = SLR_model.convert_to_dataset(x_train, y_train, batch_size)
# hist = model.fit(x_train, y_train, batch_size = 2, epochs=epochs, callbacks=[model_checkpoint_callback])

# with open(hist_path, 'w') as file:
#     json.dump(hist.history, file)


In [10]:
# plt.plot(range(len(hist.history['loss'])), hist.history['loss'])
# plt.scatter(range(len(hist.history['loss'])), hist.history['loss'])
# plt.xlabel("epochs")
# plt.ylabel("loss")
# plt.legend(["loss"])
# plt.ylim((0,1))
# plt.show()

In [11]:
# print(os.listdir(os.path.join(preprocess.getoutputdir(),str(15))))

In [12]:
# model.metrics_names